In [2]:
import pandas as pd
import xlrd
from datetime import datetime

base = pd.read_excel('D:\OneDrive\OneDrive\Scripts python\Salesforce\Owner Vs Inf. Leonardo.xlsx') #GLOBAL
cambios = pd.read_excel('D:\OneDrive\OneDrive\Scripts python\Salesforce\Leonardo2.xlsx') #CAMBIOS
cambios = cambios[cambios["Field / Event"]=="Account Owner"]
cambios['Edit Date']=cambios['Edit Date'].astype(str)

def conversion(x):
    formats_to_try = [
        "%m/%d/%Y %I:%M %p",
        "%Y-%d-%m %H:%M:%S",
        "%m/%d/%Y",
        # Agrega más formatos aquí si es necesario
    ]  
    for format_str in formats_to_try:
        try:
            date_obj = datetime.strptime(x, format_str)
            return date_obj#.strftime("%d/%m/%Y %H:%M:%S")
        except ValueError:
            pass  # Intenta el siguiente formato si este falla
    
    raise ValueError(f"Fecha en formato incorrecto: {x}")

cambios["cambiosfechas"] = cambios['Edit Date'].apply(conversion)
# Primeros Comerciales 
idxs = cambios.groupby(["Account Name"])["cambiosfechas"].agg(["idxmin", "idxmax"])
left = cambios.loc[idxs["idxmin"]]
left = left[["Account Name","Old Value","Created Date"]]
left["Created Date"] = left['Created Date'].astype(str)
left["Created Date"] = left['Created Date'].apply(conversion)
#cambios presentados
cambiosOA_New = cambios[["Account Name","New Value","cambiosfechas"]]
# Renombrar columnas
cambiosOA_New = cambiosOA_New.rename(columns={"New Value":"Sales Rep","cambiosfechas":"Date"})
left = left.rename(columns={"Old Value":"Sales Rep","Created Date":"Date"})
# Concatenar los 2 dataframes
conc_ini_act = pd.concat([left,cambiosOA_New])
# Reduccion del DF_base
base_red = base[["Account Name","Account Owner","Created Date"]]
# Renombrar columnas
left = left.rename(columns={"Account Owner":"Sales Rep","Created Date":"Date"})
# Cruzar los 2 dataframes unidos para filtrar duplicidad
# 1. Generar un dataframe con valores unicos del concatenado
unic_value = conc_ini_act[["Account Name"]].drop_duplicates()
unic_value["Constante"] = 1
# 2. realizar un merge para identificar valores que podrian repetirse
unic_value_base = base_red.merge(unic_value, on = "Account Name", how="left")
# 3. Filtrar lo que esta vacio que son los valores que no han cambiado
unic_value_base_filt = unic_value_base[unic_value_base["Constante"].isnull()]
unic_value_base_filt2 = unic_value_base_filt[unic_value_base_filt["Account Name"].notnull()]
unic_value_base_red = unic_value_base_filt2[["Account Name","Account Owner","Created Date"]]
unic_value_base_red = unic_value_base_red.rename(columns={"Account Owner":"Sales Rep","Created Date":"Date"})
unic_value_base_red["Date"] = unic_value_base_red['Date'].astype(str)
unic_value_base_red["Date"] = unic_value_base_red['Date'].apply(conversion)
maestro = pd.concat([unic_value_base_red,conc_ini_act])
maestro.sort_values(by =["Account Name","Date"], inplace=True)
# Crear una nueva columna 'nueva_fecha' basada en 'fecha_asignado' con el segundo valor
maestro['nueva_fecha'] = maestro['Date'].shift(-1)
maestro['nueva_cliente'] = maestro['Account Name'].shift(-1)

# En la última fila, establecer el valor en nulo (NaN)
maestro.at[maestro.index[-1], 'nueva_fecha'] = pd.NaT
maestro.at[maestro.index[-1], 'nueva_cliente'] = ''

def validacion(row):
    if row['Account Name'] == row['nueva_cliente']:
        result = row['nueva_fecha']
    else:
        result = pd.to_datetime('2023-12-31')
    return result

maestro['fecha_fin'] = maestro.apply(validacion, axis=1)
# Imprimir el DataFrame resultante
maestro = maestro[["Account Name","Sales Rep","Date","fecha_fin"]]

In [3]:
maestro.head()

,Account Name,Sales Rep,Date,fecha_fin
0,23KY COLOMBIA SAS,LAURA URREGO,2020-12-03,2023-12-31
1,360 GLOBAL SERVICE SAS,GLORIA GONZALEZ,2021-10-24,2023-12-31
2,360º INTEGRAL SECURITY S.A.S,CATHERINE TOQUICA,2021-04-27,2023-12-31
3,3BIOMAT,DIEGO GUZMÁN,2023-02-02,2023-12-31
4,3C COL EMERALDS C I S A S,Jairo Garcia Velasquez,2023-02-21,2023-12-31


In [4]:
maestro.to_excel("HistoryAccountSalesforce.xlsx")